In [4]:
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))


import pickle as pkl

In [5]:
with open('./datasets/singletons_af_w_pza_graph_dict.pkl', 'rb') as f:
    data = pkl.load(f)

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
len(data['test'].keys())

200

In [12]:
data['test']['pnca_mut_99']['metadata']

,phenotype_label,number_resistant_mutations,number_susceptible_mutations,allele,mutation
99,R,1,0,MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVA...,I90T


In [17]:
data['test']['pnca_mut_99']['graph'].dataset

[Data(x=[185, 16], edge_index=[2, 1840], y=1)]

In [18]:
data['test']['pnca_mut_99']['graph'].cutoff_distance

8

In [26]:
data['train']['pnca_mut_1']['graph'].dataset[0].x

tensor([[162.9000,  -0.4400, 149.2000,  ...,   0.0000,   0.0000,   0.0000],
        [173.4000,   1.0000, 174.2000,  ...,   0.0000,   0.0000,   0.0000],
        [ 88.6000,   0.3300,  89.1000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [140.0000,  -0.5300, 117.1000,  ...,   0.0000,   0.0000,   0.0000],
        [108.5000,   0.2200, 121.2000,  ...,   0.0000,   0.0000,   0.0000],
        [ 89.0000,   0.3300, 105.1000,  ...,   0.0000,   0.0000,   0.0000]])